## 1️⃣ Setup - Clone Repository & Install Dependencies

In [ ]:
# IMPORTANT: Always start fresh - remove old clone if exists
import os
import shutil

REPO_PATH = '/content/Multimodal-Rare-Disease'

# Remove old clone to start fresh
if os.path.exists(REPO_PATH):
    print("Removing old clone...")
    shutil.rmtree(REPO_PATH)

# Clone fresh
print("Cloning repository...")
!git clone --depth=1 https://github.com/ArshvirSk/Multimodal-Rare-Disease.git {REPO_PATH}

# Change to repo directory
%cd {REPO_PATH}

# Verify images are present
print(f"\n📁 Current directory: {os.getcwd()}")
print(f"\n📷 Checking images...")
!ls -la data/images/ | head -5
image_count = len([f for f in os.listdir('data/images') if f.endswith('.png')])
print(f"\n✅ Found {image_count} images in data/images/")

In [ ]:
# Install dependencies
!pip install -q torch torchvision transformers pillow tqdm matplotlib seaborn
print("✅ Dependencies installed!")

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ No GPU detected! Go to Runtime > Change runtime type > GPU")

## 2️⃣ Generate Augmented Training Data

Expands 50 base images → 550 augmented images

In [ ]:
# Verify source images exist (using absolute paths)
import os
from pathlib import Path

REPO_PATH = '/content/Multimodal-Rare-Disease'
source_dir = Path(f'{REPO_PATH}/data/images')

print(f"Source directory: {source_dir}")
print(f"Exists: {source_dir.exists()}")

if source_dir.exists():
    images = list(source_dir.glob('*.png'))
    print(f"✅ Found {len(images)} PNG images")
    print("\nSample files:")
    for img in images[:5]:
        print(f"  {img.name}")
else:
    print("❌ Source directory not found!")
    print(f"Check that {REPO_PATH} was cloned correctly")

In [ ]:
# Generate augmented images (using absolute paths)
REPO_PATH = '/content/Multimodal-Rare-Disease'
!python {REPO_PATH}/scripts/generate_augmented_data.py --source {REPO_PATH}/data/images --output {REPO_PATH}/data/images_augmented --num-aug 10

In [ ]:
# Verify augmented data
from pathlib import Path

REPO_PATH = '/content/Multimodal-Rare-Disease'
aug_dir = Path(f"{REPO_PATH}/data/images_augmented")

if aug_dir.exists():
    total_images = sum(1 for _ in aug_dir.rglob("*.png")) + sum(1 for _ in aug_dir.rglob("*.jpg"))
    syndromes = [d.name for d in aug_dir.iterdir() if d.is_dir()]

    print(f"✅ Total augmented images: {total_images}")
    print(f"✅ Syndromes: {len(syndromes)}")
    for s in sorted(syndromes):
        count = len(list((aug_dir / s).glob("*")))
        print(f"   • {s}: {count} images")
else:
    print("❌ Augmentation failed - directory not found!")

## 3️⃣ Train the Model

Training with:
- **Backbone:** ResNet50 (pretrained on ImageNet)
- **Transfer Learning:** Layers 1-3 frozen
- **Regularization:** Dropout 0.6, Label smoothing 0.12
- **Optimizer:** AdamW with Cosine Annealing LR

⏱️ Takes ~10-15 minutes on T4 GPU

In [ ]:
# Train the model
REPO_PATH = '/content/Multimodal-Rare-Disease'
!python {REPO_PATH}/run_training.py --image-dirs {REPO_PATH}/data/images_augmented --epochs 60 --aug-factor 4 --batch-size 16

## 🚀 Alternative: Use Pre-trained Model (Skip Training)

**Don't want to wait 10-15 minutes?** Download our pre-trained model instead!

This model was trained on 550 augmented images and achieves ~95% validation accuracy.

In [ ]:
# Download pre-trained model (skip training!)
import os
import urllib.request

REPO_PATH = '/content/Multimodal-Rare-Disease'

# Create checkpoints directory
os.makedirs(f'{REPO_PATH}/checkpoints', exist_ok=True)

# Pre-trained model URL (hosted on GitHub Releases)
MODEL_URL = "https://github.com/ArshvirSk/Multimodal-Rare-Disease/releases/download/v1.0/best_model.pt"
MODEL_PATH = f"{REPO_PATH}/checkpoints/best_model.pt"

if not os.path.exists(MODEL_PATH):
    print("📥 Downloading pre-trained model (~221 MB)...")
    try:
        urllib.request.urlretrieve(MODEL_URL, MODEL_PATH)
        print(f"✅ Model downloaded to {MODEL_PATH}")
    except Exception as e:
        print(f"⚠️ Download failed: {e}")
        print("\nAlternative: Train the model using the cell above (takes ~10-15 min)")
else:
    print(f"✅ Model already exists at {MODEL_PATH}")

# Verify model
if os.path.exists(MODEL_PATH):
    size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)
    print(f"📦 Model size: {size_mb:.1f} MB")

## 4️⃣ Make Predictions

Test the trained model on sample images

In [ ]:
# Test prediction on a sample image
from pathlib import Path

REPO_PATH = '/content/Multimodal-Rare-Disease'
aug_dir = Path(f"{REPO_PATH}/data/images_augmented")

test_imgs = list(aug_dir.rglob("*_orig.png"))
if test_imgs:
    test_img = test_imgs[0]
    print(f"Testing on: {test_img}\n")
    !python {REPO_PATH}/predict.py --image "{test_img}"
else:
    print("No test images found. Run augmentation first.")

In [ ]:
# Test on multiple syndromes
from pathlib import Path

REPO_PATH = '/content/Multimodal-Rare-Disease'
aug_dir = Path(f"{REPO_PATH}/data/images_augmented")

syndromes = sorted([d for d in aug_dir.iterdir() if d.is_dir()])[:4]

for syn_dir in syndromes:
    orig_imgs = list(syn_dir.glob("*_orig.png"))
    if orig_imgs:
        orig_img = orig_imgs[0]
        print(f"\n{'='*60}")
        print(f"Testing: {syn_dir.name}")
        !python {REPO_PATH}/predict.py --image "{orig_img}"

## 5️⃣ Upload Your Own Image

Upload a facial image to classify

In [ ]:
# Upload your own image
from google.colab import files
from IPython.display import Image, display
import os
import shutil

REPO_PATH = '/content/Multimodal-Rare-Disease'

print("📤 Upload a facial image (JPG/PNG):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    # Copy to repo directory
    shutil.copy(filename, f"{REPO_PATH}/{filename}")
    print(f"\n✅ Uploaded: {filename}")
    
    # Display the image
    display(Image(filename, width=300))
    
    # Run prediction
    print("\n🔍 Running prediction...\n")
    !python {REPO_PATH}/predict.py --image "{REPO_PATH}/{filename}"

## 6️⃣ Visualize Sample Images

In [ ]:
# Visualize sample images from each syndrome
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path

REPO_PATH = '/content/Multimodal-Rare-Disease'
aug_dir = Path(f"{REPO_PATH}/data/images_augmented")

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Images from Each Syndrome', fontsize=14)

syndromes = sorted([d for d in aug_dir.iterdir() if d.is_dir()])

syndrome_names = {
    "SYN_CdLS": "Cornelia de Lange",
    "SYN_WBS": "Williams-Beuren",
    "SYN_NS": "Noonan",
    "SYN_KS": "Kabuki",
    "SYN_KBG": "KBG",
    "SYN_AS": "Angelman",
    "SYN_RSTS": "Rubinstein-Taybi",
    "SYN_SMS": "Smith-Magenis",
    "SYN_NBS": "Nicolaides-Baraitser",
    "SYN_22Q": "22q11.2 Deletion",
}

for idx, syn_dir in enumerate(syndromes[:10]):
    ax = axes[idx // 5, idx % 5]
    orig_imgs = list(syn_dir.glob("*_orig.png"))
    if orig_imgs:
        img = Image.open(orig_imgs[0])
        ax.imshow(img)
    name = syndrome_names.get(syn_dir.name, syn_dir.name)
    ax.set_title(name, fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 📊 Model Architecture

```
┌─────────────────────────────────────────┐
│           Input Image (224x224)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│     ResNet50 Backbone (ImageNet)        │
│     • Layers 1-3: Frozen                │
│     • Layer 4: Trainable                │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│         Feature Vector (2048-d)         │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│    Classification Head + Dropout        │
└─────────────────┬───────────────────────┘
                  ▼
┌─────────────────────────────────────────┐
│      Softmax → 10 Syndrome Classes      │
└─────────────────────────────────────────┘
```

---

## 🔗 Links

- **GitHub:** https://github.com/ArshvirSk/Multimodal-Rare-Disease
- **PDIDB Dataset:** https://github.com/WGLab/PDIDB
- **Orphadata:** https://www.orphadata.com

---

*Made with ❤️ for rare disease research*